In [1]:
import openai 
import random
import pandas as pd

# Determine the model: can be gpt-3.5-turbo or gpt-4
model_name = "gpt-3.5-turbo"

In [2]:
try:
  api_key = "INSERT KEY" # Add API key
  openai.api_key = api_key
  print("API key successfully loaded.")
except:
  print("No API key file found.")

API key successfully loaded.


In [3]:
# CSV path (change accordingly)
csv_path = "test_C.csv" 
prompts = []
fields = ['Instructions', 'Condition', 'Study', 'Type', 'Proposition', 'NAND', 'NOR', 'Positive AND']  # Columns used in CSV

# Reading CSV to dataframe
prompts_df = pd.read_csv(csv_path, usecols=fields)

for _, row in prompts_df.iterrows():
    prompt = {}

    # Check if either "A" or "B" exists in the row
    choices = {"NAND":row["NAND"], "NOR":row["NOR"], "Contradictory":row["Positive AND"]}
    
    text_parts = [row['Instructions'], row['Proposition']]

    
    # Shuffle the keys of choices
    shuffled_keys = list(choices.keys())
    random.shuffle(shuffled_keys)

    labels = ["A", "B", "C"]

    # Iterate through shuffled keys and assign values to text_parts
    contradictory_index = None
    for label, key in zip(labels, shuffled_keys):
        if key == "Contradictory":
            contradictory_index = len(text_parts)
            contradictory_label = label
        text_parts.extend([label + ":", choices[key]])
    
    # Filter out empty strings before joining
    prompt["text"] = ' '.join(filter(None, text_parts))
    prompt["condition"] = row['Condition']
    prompt["study"] = row['Study']
    prompt["type"] = row['Type']
    prompt["contrar"] = contradictory_label
    
    prompts.append(prompt)

random.shuffle(prompts)

print(f"Found {len(prompts)} prompts in the given CSV file.")

Found 8 prompts in the given CSV file.


In [4]:
outputs = []
total_tokens = 0

# Calling chatGPT and appending responses to outputs list
for prompt in prompts:
  response = openai.ChatCompletion.create(
                model=model_name,
                messages=[
                      {"role": "system", "content": "Respond with only either 'A', 'B', or 'C' and nothing else."},
                      {"role": "user", "content": prompt["text"]},
                  ]
              )
  gpt_answer = response['choices'][0]['message']['content']
  outputs.append(gpt_answer)

  total_tokens += response["usage"]["total_tokens"]

print(f"Messages generated with {total_tokens} tokens")

Messages generated with 821 tokens


In [5]:
df = pd.DataFrame({"Prompt": [prompt["text"] for prompt in prompts],  
                   "Condition": [prompt["condition"] for prompt in prompts],
                   "Study": [prompt["study"] for prompt in prompts],
                   "Type": [prompt["type"] for prompt in prompts],
                   "Contradictory_cont": [prompt["contrar"] for prompt in prompts],
                   "Response": outputs})

# Save the DataFrame to a CSV file
output_csv_path = "output_responses_C.csv"
df.to_csv(output_csv_path, index=False)

print(f"Responses saved to {output_csv_path}")

Responses saved to output_responses_C.csv
